### Creating a graph using networkx

In [ ]:
pip install scipy==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.6 MB 1.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [ ]:
pip install networkx==2.8.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.2 MB/s 
  Attempting uninstall: networkx
    Found existing installation: networkx 2.8.8
    Uninstalling networkx-2.8.8:
      Successfully uninstalled networkx-2.8.8


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random
import operator
import math
import heapq
import itertools

In [ ]:
G = nx.read_edgelist('facebook_combined.txt')

In [ ]:
print(list(G.edges)[0])
print(list(G.edges)[2])

('0', '1')
('0', '3')


In [ ]:
G_orig = G.copy()
GBet = G.copy()

In [ ]:
nc = G.number_of_nodes()
print("Number of nodes: ",nc)
ne = G.number_of_edges()
print("Number of edges: ",ne)

Number of nodes:  4039
Number of edges:  88234


### Adjacency matrix of the graph

In [ ]:
A = nx.adjacency_matrix(G).todense()

print(A)

<ipython-input-14-b8d2a4bd5802>:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G).todense()


[[0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
adj_mat = []
for i in A:
    adj_mat.append(list(i.A[0]))

for i in range(nc):
  for j in range(nc):
    if adj_mat[i][j] > 0:
      adj_mat[i][j] = 1

### Local graph sparsification

In [ ]:
def create_empty_copy(G, with_data=True):
    H = G.__class__()
    H.add_nodes_from(G.nodes(data=with_data))
    if with_data:
        H.graph.update(G.graph)
    return H

In [ ]:
sparse_graph = create_empty_copy(G,with_data=True)

In [ ]:
degree = {}
for node in G.nodes:
  degree[node] = G.degree[node]

In [ ]:
adj_list = {}
for node in G.nodes:
  adj_list[node] = []
  for n in G.neighbors(node):
    adj_list[node].append(n)

In [ ]:
print(adj_list)

{'0': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '1

In [ ]:
def similarity(node1,node2):
 adj1 = adj_list[node1]
 adj2 = adj_list[node2]
 sim_val = len(set(adj1) & set(adj2))/len(set(adj1) | set(adj2))
 return sim_val

In [ ]:
# local sparsification algorithm
# local sparsification exponent e = 0.75
e = 0.75
for node in G.nodes:
  d = degree[node]
  adj = adj_list[node]
  i = 1
  sim = {}
  thresh = math.floor(pow(d,e))

  for i in adj:
    sim[i] = similarity(node,i)
  sorted_sim = dict( sorted(sim.items(), key=operator.itemgetter(1),reverse=True))
  sorted_sim = dict(itertools.islice(sorted_sim.items(), thresh))
  for i in sorted_sim.keys():
    if not sparse_graph.has_edge(node,i):
      sparse_graph.add_edge(node,i)
       

In [ ]:
nsc = sparse_graph.number_of_nodes()
print("Number of nodes: ",nsc)
nse = sparse_graph.number_of_edges()
print("Number of edges: ",nse)

Number of nodes:  4039
Number of edges:  43432


### Find the average degree of the graph

In [ ]:
avg_degree = 0
sum_of_degree = 0

if nc < 10000:
  for i in range(nc):
    temp_sum = 0
    for j in range(nc):
      temp_sum += adj_mat[i][j]
    sum_of_degree += temp_sum
  
else:
  k = 10000
  sampled_nodes = random.sample(G.nodes, k)
  sampled_graph = G.subgraph(sampled_nodes)
  for node in sampled_graph.nodes:
    sum_of_degree += sampled_graph.degree[node]

avg_degree = sum_of_degree//nc
print(avg_degree)

43


### Find the connected components in the sparsified graph

In [ ]:
# identify the edges with degree higher than average degree and divide the graph into components based on those edges
adj_slist = {}
for node in sparse_graph.nodes:
  adj_slist[node] = []
  for n in sparse_graph.neighbors(node):
    adj_slist[node].append(n)

In [ ]:
hc_nodes_dict = {}
hc_nodes = []
for node in sparse_graph:
  if sparse_graph.degree[node] > avg_degree:
    hc_nodes_dict[node] = sparse_graph.degree[node]
hc_nodes_dict = dict( sorted(hc_nodes_dict.items(), key=operator.itemgetter(1),reverse=True))
for key in hc_nodes_dict.keys():
  hc_nodes.append(key)
print(hc_nodes)


['107', '2090', '2059', '2586', '2220', '2073', '2464', '2340', '2201', '2078', '2244', '2030', '2206', '2290', '1684', '2088', '1962', '2507', '1912', '2604', '1993', '2118', '1938', '2131', '2275', '2218', '2123', '2601', '2593', '2625', '1946', '2240', '2590', '2624', '2369', '3437', '1943', '2104', '2188', '2615', '2309', '1917', '2187', '2139', '2607', '2064', '2087', '2150', '2356', '1589', '2326', '2331', '2602', '2153', '2598', '1516', '1833', '1612', '1945', '2032', '2271', '2410', '1199', '1376', '2564', '2138', '2172', '2323', '1621', '1746', '2463', '2354', '1707', '1827', '1059', '1078', '1559', '1888', '0', '1185', '1714', '2414', '2500', '1352', '1663', '1768', '1603', '1804', '2224', '2471', '1126', '2184', '2428', '1835', '2229', '1211', '1390', '2045', '1377', '1431', '2103', '1730', '1800', '2510', '2630', '1399', '1551', '2111', '2555', '2140', '2381', '1799', '1184', '2183', '2283', '2629', '1984', '1367', '1557', '1577', '2071', '2279', '2108', '2655', '2395', '24

In [ ]:
nodes_left = list(sparse_graph.nodes)
for i in hc_nodes:
  if i in nodes_left:
    nodes_left.remove(i)
print(nodes_left)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', 

In [ ]:

components = []

for node in hc_nodes:
  component = []
  component.append(node) 
  components.append(component)

In [ ]:
visited = {}
for node in nodes_left:
  visited[node] = False
for node in hc_nodes:
  visited[node] = True
for component in components:
  for i in component:
    for j in adj_slist[i]:
      if not visited[j]:
        component.append(j)
        nodes_left.remove(j)
        visited[j] = True

In [ ]:
print(len(components))

511


In [ ]:
subgraphs = []
for component in components:
  temp = nx.Graph()
  temp = sparse_graph.subgraph(component)
  subgraphs.append(temp)

### Construct a super graph

In [ ]:
all_edges = list(sparse_graph.edges)

In [ ]:
print(len(all_edges))

43432


In [ ]:
sg_edges = []
for graph in subgraphs:
  for edge in graph.edges:
    sg_edges.append(edge)

In [ ]:
print(len(sg_edges))

20127


In [ ]:
map_to_component = {}
for node in hc_nodes:
  for component in components:
    if node in component:
      map_to_component[node] = component
print(map_to_component)

{'107': ['107', '917', '896', '1277', '1783', '1849', '1617', '1597', '1191', '953', '1104', '1204', '1302', '1864', '1409', '1530', '1886', '978', '1741', '1789', '1149', '1459', '1810', '1826', '1845', '1334', '1339', '1643', '1014', '1156', '1637', '1861', '1128', '1192', '1786', '1483', '1003', '911', '918', '1096', '1119', '1145', '1206', '1386', '1466', '1560', '1581', '1834', '1373', '1742', '942', '1871', '1166', '1066', '1661', '1894', '1828', '1751', '1454', '1108', '1515', '1876', '1732', '1582', '1273', '1382', '954', '1893', '898', '1245', '1502', '1159', '1303', '1536', '1573', '1227', '1231', '1591', '1347', '957', '1475', '1426', '1021', '1158', '1413', '1691', '1036', '1129', '1009', '1417', '1875', '1221', '1716', '1082', '1423', '1148', '1720', '1495', '984', '1812', '1794', '1351', '1580', '952', '1028', '1092', '1287', '1293', '1700', '1181', '1340', '1632', '1653', '1737', '1819', '1877', '1056', '1074', '1609', '1323', '1393', '1407', '1811', '932', '1160', '1359

In [ ]:
super_graph = nx.Graph()
super_graph.add_nodes_from(hc_nodes)
for node in super_graph.nodes:
  comp_list = map_to_component[node]
  temp = []
  temp.extend(hc_nodes)
  temp.remove(node)
  for i in comp_list:
    for j in adj_list[i] :
      if j in temp and sparse_graph.has_edge(node,j):
        super_graph.add_edge(node,j)
print(len(super_graph.edges))

12791


In [ ]:
print(len(super_graph.edges))

12791


In [ ]:
import warnings
from collections import deque
from heapq import heappop, heappush
from itertools import count

from networkx.algorithms.shortest_paths.weighted import _weight_function
from networkx.utils import py_random_state
from networkx.utils.decorators import not_implemented_for


In [ ]:
# helper functions

def _single_source_shortest_path_basic(G, s):
    S = []
    P = {}
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)  # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    D[s] = 0
    Q = deque([s])
    while Q:  # use BFS to find shortest paths
        v = Q.popleft()
        S.append(v)
        Dv = D[v]
        sigmav = sigma[v]
        for w in G[v]:
            if w not in D:
                Q.append(w)
                D[w] = Dv + 1
            if D[w] == Dv + 1:  # this is a shortest path, count paths
                sigma[w] += sigmav
                P[w].append(v)  # predecessors
    return S, P, sigma, D


def _single_source_dijkstra_path_basic(G, s, weight):
    weight = _weight_function(G, weight)
    # modified from Eppstein
    S = []
    P = {}
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)  # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    push = heappush
    pop = heappop
    seen = {s: 0}
    c = count()
    Q = []  # use Q as heap with (distance,node id) tuples
    push(Q, (0, next(c), s, s))
    while Q:
        (dist, _, pred, v) = pop(Q)
        if v in D:
            continue  # already searched this node.
        sigma[v] += sigma[pred]  # count paths
        S.append(v)
        D[v] = dist
        for w, edgedata in G[v].items():
            vw_dist = dist + weight(v, w, edgedata)
            if w not in D and (w not in seen or vw_dist < seen[w]):
                seen[w] = vw_dist
                push(Q, (vw_dist, next(c), v, w))
                sigma[w] = 0.0
                P[w] = [v]
            elif vw_dist == seen[w]:  # handle equal paths
                sigma[w] += sigma[v]
                P[w].append(v)
    return S, P, sigma, D


def _accumulate_basic(betweenness, S, P, sigma, s):
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1 + delta[w]) / sigma[w]
        for v in P[w]:
            delta[v] += sigma[v] * coeff
        if w != s:
            betweenness[w] += delta[w]
    return betweenness, delta


def _accumulate_endpoints(betweenness, S, P, sigma, s):
    betweenness[s] += len(S) - 1
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1 + delta[w]) / sigma[w]
        for v in P[w]:
            delta[v] += sigma[v] * coeff
        if w != s:
            betweenness[w] += delta[w] + 1
    return betweenness, delta


def _accumulate_edges(betweenness, S, P, sigma, s):
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1 + delta[w]) / sigma[w]
        for v in P[w]:
            c = sigma[v] * coeff
            if (v, w) not in betweenness:
                betweenness[(w, v)] += c
            else:
                betweenness[(v, w)] += c
            delta[v] += c
        if w != s:
            betweenness[w] += delta[w]
    return betweenness


def _rescale(betweenness, n, normalized, directed=False, k=None, endpoints=False):
    if normalized:
        if endpoints:
            if n < 2:
                scale = None  # no normalization
            else:
                # Scale factor should include endpoint nodes
                scale = 1 / (n * (n - 1))
        elif n <= 2:
            scale = None  # no normalization b=0 for all nodes
        else:
            scale = 1 / ((n - 1) * (n - 2))
    else:  # rescale by 2 for undirected graphs
        if not directed:
            scale = 0.5
        else:
            scale = None
    if scale is not None:
        if k is not None:
            scale = scale * n / k
        for v in betweenness:
            betweenness[v] *= scale
    return betweenness


def _rescale_e(betweenness, n, normalized, directed=False, k=None):
    if normalized:
        if n <= 1:
            scale = None  # no normalization b=0 for all nodes
        else:
            scale = 1 / (n * (n - 1))
    else:  # rescale by 2 for undirected graphs
        if not directed:
            scale = 0.5
        else:
            scale = None
    if scale is not None:
        if k is not None:
            scale = scale * n / k
        for v in betweenness:
            betweenness[v] *= scale
    return betweenness


@not_implemented_for("graph")
def _add_edge_keys(G, betweenness, weight=None):
    
    _weight = _weight_function(G, weight)

    edge_bc = dict.fromkeys(G.edges, 0.0)
    for u, v in betweenness:
        d = G[u][v]
        wt = _weight(u, v, d)
        keys = [k for k in d if _weight(u, v, {k: d[k]}) == wt]
        bc = betweenness[(u, v)] / len(keys)
        for k in keys:
            edge_bc[(u, v, k)] = bc

    return edge_bc

In [ ]:
@py_random_state(4)
def edge_betweenness_centrality(G, k=None, normalized=True, weight=None, seed=None):
    
    betweenness = dict.fromkeys(G, 0.0)  # b[v]=0 for v in G
    # b[e]=0 for e in G.edges()
    betweenness.update(dict.fromkeys(G.edges(), 0.0))
    if k is None:
        nodes = G
    else:
        nodes = seed.sample(G.nodes(), k)
    for s in nodes:
        # single source shortest paths
        if weight is None:  # use BFS
            S, P, sigma, _ = _single_source_shortest_path_basic(G, s)
        else:  
            S, P, sigma, _ = _single_source_dijkstra_path_basic(G, s, weight)
        # accumulation
        betweenness = _accumulate_edges(betweenness, S, P, sigma, s)
    # rescaling
    for n in G:  # remove nodes to only return edges
        del betweenness[n]
    betweenness = _rescale_e(
        betweenness, len(G), normalized=normalized, directed=G.is_directed()
    )
    if G.is_multigraph():
        betweenness = _add_edge_keys(G, betweenness, weight=weight)
    return betweenness


In [ ]:
spg_edge_bc = edge_betweenness_centrality(super_graph)

In [ ]:
print(spg_edge_bc)

{('107', '483'): 0.03729711062507195, ('107', '916'): 0.00043633427227895353, ('107', '921'): 0.0006689771450043793, ('107', '925'): 0.0005310983803267324, ('107', '946'): 0.00045207950051754823, ('107', '947'): 0.0005956806763544728, ('107', '960'): 0.0008946148015720772, ('107', '966'): 0.0006249234054619693, ('107', '967'): 0.0006712644674399724, ('107', '980'): 0.0005853505436774408, ('107', '993'): 0.0004892753227738934, ('107', '997'): 0.0007380887129361863, ('107', '1004'): 0.0007204547745181802, ('107', '1006'): 0.000609158601073252, ('107', '1017'): 0.00045052852737509483, ('107', '1048'): 0.0006790428313913973, ('107', '1059'): 0.0005371125032077491, ('107', '1076'): 0.000562835405921793, ('107', '1078'): 0.00039950653190522857, ('107', '1079'): 0.0006967824274862173, ('107', '1083'): 0.0009095533593343124, ('107', '1086'): 0.00041399287666074526, ('107', '1101'): 0.0005695531133454735, ('107', '1107'): 0.0004737577260368155, ('107', '1117'): 0.000539634502265546, ('107', '11

### Scale the edge betweenness centrality using a heuristic based on number of vertices in connected components

In [ ]:
map_to_component = {}
for node in hc_nodes:
  for component in components:
    if node in component:
      map_to_component[node] = component
print(map_to_component)

{'107': ['107', '917', '896', '1277', '1783', '1849', '1617', '1597', '1191', '953', '1104', '1204', '1302', '1864', '1409', '1530', '1886', '978', '1741', '1789', '1149', '1459', '1810', '1826', '1845', '1334', '1339', '1643', '1014', '1156', '1637', '1861', '1128', '1192', '1786', '1483', '1003', '911', '918', '1096', '1119', '1145', '1206', '1386', '1466', '1560', '1581', '1834', '1373', '1742', '942', '1871', '1166', '1066', '1661', '1894', '1828', '1751', '1454', '1108', '1515', '1876', '1732', '1582', '1273', '1382', '954', '1893', '898', '1245', '1502', '1159', '1303', '1536', '1573', '1227', '1231', '1591', '1347', '957', '1475', '1426', '1021', '1158', '1413', '1691', '1036', '1129', '1009', '1417', '1875', '1221', '1716', '1082', '1423', '1148', '1720', '1495', '984', '1812', '1794', '1351', '1580', '952', '1028', '1092', '1287', '1293', '1700', '1181', '1340', '1632', '1653', '1737', '1819', '1877', '1056', '1074', '1609', '1323', '1393', '1407', '1811', '932', '1160', '1359

In [ ]:
for edge in spg_edge_bc:
  spg_edge_bc[edge] = len(map_to_component[edge[0]]) * len(map_to_component[edge[1]]) * spg_edge_bc[edge]

In [ ]:
print(spg_edge_bc)

{('107', '483'): 84.66444111891332, ('107', '916'): 0.9904787980732245, ('107', '921'): 1.518578119159941, ('107', '925'): 1.2055933233416825, ('107', '946'): 1.0262204661748344, ('107', '947'): 1.3521951353246533, ('107', '960'): 2.0307755995686154, ('107', '966'): 1.4185761303986704, ('107', '967'): 1.5237703410887375, ('107', '980'): 1.3287457341477906, ('107', '993'): 1.110654982696738, ('107', '997'): 1.675461378365143, ('107', '1004'): 1.635432338156269, ('107', '1006'): 1.382790024436282, ('107', '1017'): 1.0226997571414653, ('107', '1048'): 1.541427227258472, ('107', '1059'): 1.2192453822815905, ('107', '1076'): 1.27763637144247, ('107', '1078'): 0.9068798274248688, ('107', '1079'): 1.5816961103937133, ('107', '1083'): 2.064686125688889, ('107', '1086'): 0.9397638300198917, ('107', '1101'): 1.2928855672942248, ('107', '1107'): 1.0754300381035713, ('107', '1117'): 1.2249703201427893, ('107', '1124'): 1.1398977430357304, ('107', '1125'): 1.4606592711837285, ('107', '1126'): 0.910

In [ ]:
print(len(spg_edge_bc))

12791


In [ ]:
# Budget ofedges to be removed
k = 15000

In [ ]:
if k < len(spg_edge_bc):
  spg_edge_bc = dict( sorted(spg_edge_bc.items(), key=operator.itemgetter(1),reverse=True))
  spg_edge_bc = dict(itertools.islice(spg_edge_bc.items(), k))

### Reduction of budget of edges to be removed

In [ ]:

# Budget reduced after removing edges from super graph

k = k - len(spg_edge_bc)

if k < 0:
  k = 0

print("Budget left :", k)

Budget left : 2209


### Split the budget between the components

In [ ]:
comp_budget = {}
for graph in subgraphs:
  comp_budget[graph] = 0.5 * len(graph.nodes) + 0.5 * len(graph.edges)
sum = 0

for ind in comp_budget.values():
  sum += ind

result = {key: round(value * k / sum) for key, value in comp_budget.items()}  

i = 1
for j in result.values():
  print(i,"th component budget : ",j)
  i = i + 1

1 th component budget :  1533
2 th component budget :  322
3 th component budget :  0
4 th component budget :  0
5 th component budget :  0
6 th component budget :  0
7 th component budget :  0
8 th component budget :  0
9 th component budget :  0
10 th component budget :  0
11 th component budget :  0
12 th component budget :  0
13 th component budget :  0
14 th component budget :  0
15 th component budget :  1
16 th component budget :  0
17 th component budget :  0
18 th component budget :  0
19 th component budget :  1
20 th component budget :  0
21 th component budget :  0
22 th component budget :  0
23 th component budget :  0
24 th component budget :  0
25 th component budget :  0
26 th component budget :  0
27 th component budget :  0
28 th component budget :  0
29 th component budget :  0
30 th component budget :  0
31 th component budget :  0
32 th component budget :  0
33 th component budget :  0
34 th component budget :  0
35 th component budget :  0
36 th component budget :

### Find the edges betweenness centrality for each component using multithreading

In [ ]:
thread_count = len(subgraphs)
print(thread_count)

511


In [ ]:
import datetime
import time
import threading

class add_to_dict(threading.Thread):

    def __init__(self, lock, graph, kn, edge_list):
        threading.Thread.__init__(self)
        self.totalAdded = 0
        self.edge_list = edge_list
        self.lock = lock
        self.graph = graph
        self.kn = kn

    def run(self):        
            self.lock.acquire()
            if self.kn > 0:
              edge_bc = edge_betweenness_centrality(graph)
              sorted_edge_bc = dict( sorted(edge_bc.items(), key=operator.itemgetter(1),reverse=True))
              sorted_edge_bc2 = dict(itertools.islice(sorted_edge_bc.items(), self.kn))
              edge_list.update(sorted_edge_bc2)
            self.totalAdded += 1
            self.lock.release()                    


if __name__=="__main__":
    lock=threading.Lock()
    edge_list={}

    threads = [None] * thread_count
    start = time.time()
    i = 0
    for graph in subgraphs:
      threads[i] = add_to_dict(lock,graph,result[graph],edge_list)
      threads[i].start()
      i += 1

    for j in range(thread_count):
      threads[j].join()
    end = time.time()
    print('time taken using multi-threading : ',end-start)
    

time taken using multi-threading :  162.61847758293152


In [ ]:
edge_list.update(spg_edge_bc)

In [ ]:
removable_edges = []
for key in edge_list.keys():
  removable_edges.append(key)
print(removable_edges)

[('1085', '1098'), ('1098', '1505'), ('3097', '3117'), ('1405', '3168'), ('1405', '1637'), ('1085', '3684'), ('1085', '3609'), ('3609', '3779'), ('3779', '3941'), ('107', '1637'), ('526', '1528'), ('3684', '3496'), ('3168', '2730'), ('1085', '1360'), ('1085', '1555'), ('3097', '3340'), ('3612', '3941'), ('1505', '2764'), ('1505', '3011'), ('2764', '3340'), ('3097', '2730'), ('1348', '1847'), ('2928', '3117'), ('3496', '3521'), ('649', '1023'), ('1371', '1758'), ('1405', '3295'), ('1678', '1847'), ('3625', '3978'), ('2759', '3383'), ('3914', '3978'), ('3677', '3839'), ('1085', '1687'), ('1023', '1687'), ('389', '1348'), ('1085', '3797'), ('3504', '3839'), ('1678', '1726'), ('1553', '1555'), ('1360', '1553'), ('1085', '1588'), ('1360', '1847'), ('1555', '1847'), ('3549', '3566'), ('649', '1694'), ('1085', '3633'), ('1588', '1847'), ('3723', '3504'), ('3496', '3505'), ('1283', '1523'), ('3723', '3566'), ('2883', '3043'), ('526', '430'), ('1085', '3495'), ('1182', '1528'), ('927', '1528'),

### Graph obtained after removal of edges

In [ ]:
for edge in removable_edges:
  G.remove_edge(edge[0],edge[1])

### Set of removable edges obtained by applying normal method using edge betweenness centrality

In [ ]:
import time
k1 = 15000
start = time.time()
b=nx.edge_betweenness_centrality(GBet)
end = time.time()
print('time taken for execution : ',end-start)
sorted_b = dict( sorted(b.items(), key=operator.itemgetter(1),reverse=True))
sorted_b = dict(itertools.islice(sorted_b.items(), k1))
for edge in sorted_b.keys():
    GBet.remove_edge(edge[0],edge[1])

time taken for execution :  345.6582295894623


In [ ]:
removable_edges_exact = []
for key in sorted_b.keys():
  removable_edges_exact.append(key)

In [ ]:
# The percentage of edges that are correctly obtained using the proposed algorithm
common_list = (list(set(removable_edges_exact) & set(removable_edges)))
print(common_list)

[('1376', '1761'), ('1684', '2750'), ('171', '217'), ('389', '551'), ('1505', '2968'), ('951', '1445'), ('1420', '1522'), ('1577', '2649'), ('458', '497'), ('3851', '3596'), ('107', '1132'), ('596', '1022'), ('651', '591'), ('526', '1688'), ('1912', '2369'), ('389', '457'), ('1912', '2308'), ('1588', '1678'), ('1684', '3291'), ('171', '904'), ('2266', '2404'), ('1085', '1719'), ('107', '1017'), ('3756', '3596'), ('107', '1610'), ('1375', '1688'), ('526', '1049'), ('483', '1396'), ('1684', '2925'), ('483', '438'), ('1534', '2764'), ('3577', '3938'), ('3872', '3575'), ('107', '1793'), ('366', '515'), ('107', '1083'), ('1912', '1941'), ('107', '1096'), ('107', '925'), ('475', '484'), ('500', '503'), ('107', '1079'), ('1405', '1513'), ('1172', '1361'), ('1189', '1847'), ('107', '1192'), ('1013', '1035'), ('1024', '1863'), ('596', '1233'), ('389', '505'), ('107', '1456'), ('1266', '1666'), ('2347', '2233'), ('107', '1861'), ('475', '548'), ('990', '2760'), ('359', '546'), ('107', '1623'), (

In [ ]:
from numpy.linalg import matrix_rank,eig

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def Connectivity(graph):
  A = nx.adjacency_matrix(graph)
  r = matrix_rank(A.todense())
  vals,v = eig(A.todense())
  # vals = nx.adjacency_spectrum(graph)
  sorted(vals,reverse = True)
  vals = vals[:r]
  # Using natural connectivity as the connectivity measure
  sum = 0
  for val in vals:
    sum += math.exp(val)
  nc = sum/len(vals)
  return nc

In [ ]:
print(Connectivity(G_orig))

8.336056958566492e+66


In [ ]:
print(Connectivity(G))

2.72395729828329e+40


In [ ]:
print(Connectivity(GBet))

1.242870719384954e+64


### Jaccard's similarity of graphs

In [ ]:
adj_list_g = {}
for node in G.nodes:
  adj_list_g[node] = []
  for n in G.neighbors(node):
    adj_list_g[node].append(n)

In [ ]:
adj_list_gbet = {}
for node in GBet.nodes:
  adj_list_gbet[node] = []
  for n in GBet.neighbors(node):
    adj_list_gbet[node].append(n)

In [ ]:
def j_similarity(i,j):
 adj1 = adj_list_g[i]
 adj2 = adj_list_g[j]
 sim_val = len(set(adj1) & set(adj2))/len(set(adj1) | set(adj2))
 return sim_val

In [ ]:
def jb_similarity(i,j):
 adj1 = adj_list_gbet[i]
 adj2 = adj_list_gbet[j]
 sim_val = len(set(adj1) & set(adj2))/len(set(adj1) | set(adj2))
 return sim_val

In [ ]:
sim_g = 0
sim_gbet = 0
for edge in G.edges():
  temp = j_similarity(edge[0],edge[1])
  sim_g += temp

for edge in GBet.edges():
  temp = jb_similarity(edge[0],edge[1])
  sim_gbet += temp

print('For graph obtained through parallel computation',sim_g)
print('For graph obtained by normal method',sim_gbet)

For graph obtained through parallel computation 18009.915811143692
For graph obtained by normal method 29323.71505167765
